# Código Auxiliar para a Segunda Avaliação.

In [4]:
import pandas as pd
import sqlite3
from collections import defaultdict

As funções definidas abaixo, devem ser usada para acessar o banco de dados `bilheteria.db` e retornar os dados necessários para a resolução dos exercícios.


In [2]:
def carrega_tabela(database, tabela):
    '''
    Carrega uma tabela de um banco de dados 
    SQLite em um DataFrame do Pandas.
    Parâmetros:
    database: str - Caminho para o arquivo do banco de dados SQLite.
    tabela: str - Nome da tabela a ser carregada.
    '''
    with sqlite3.connect(database) as conn:
        query = f"SELECT * FROM {tabela}"
        df = pd.read_sql_query(query, conn)
    return df

In [3]:
def lista_tabelas(db_filename):
    '''
    Lista todas as tabelas em um banco de dados SQLite.
    Parâmetros:
    db_filename: str - Caminho para o arquivo do banco de dados SQLite.
    Retorna:
    DataFrame com os nomes das tabelas e suas contagens de linhas.
    '''
    # Connect to the SQLite database
    # conn = sqlite3.connect(db_filename)
    with sqlite3.connect(db_filename) as conn:
        cursor = conn.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()

        table_row_counts = []
        for table in tables:
            table_name = table[0]
            query = f"SELECT COUNT(*) FROM {table_name};"
            cursor.execute(query)
            row_count = cursor.fetchone()[0]
            table_row_counts.append({"Table": table_name, "Row_Count": row_count})

        return pd.DataFrame(table_row_counts)

## Exemplos de uso das funções

In [10]:
lista_tabelas('bilheteria.db')

,Table,Row_Count
0,distribuidora,72
1,filme,515
2,grupo_exibidor,64
3,exibidor,180
4,complexo,683
5,sala,3231
6,sessao,1748363


# Utilice a celula abaixo para visualizar os dados

comente e descomente as tabelas que voce quer visualizar

In [ ]:
# distribuidora = carrega_tabela("bilheteria.db", "distribuidora")
# filme = carrega_tabela("bilheteria.db", "filme")
# grupo_exibidor = carrega_tabela("bilheteria.db", "grupo_exibidor")
# exibidor = carrega_tabela("bilheteria.db", "exibidor")
# complexo = carrega_tabela("bilheteria.db", "complexo")
# sala = carrega_tabela("bilheteria.db", "sala")
# sessao = carrega_tabela('bilheteria.db', 'sessao')

complexo = carrega_tabela("bilheteria.db", "complexo")
sala = carrega_tabela("bilheteria.db", "sala")
sessao = carrega_tabela('bilheteria.db', 'sessao')
#sessao.data_exibicao = pd.to_datetime(sessao.data_exibicao, format='%d/%m/%Y')

# print(distribuidora)
# print(filme)
# print(filme.columns)
# print(grupo_exibidor)
# print(exibidor)
print(complexo)
print(sala)
print(sessao)


# Questao 1
1. Qual o total de bilheteria de todos os filmes, ou seja, o público que foi aos
filmes listados?

In [ ]:
def questao_1(database: str):
    sessao = carrega_tabela(database, "sessao")
    total_bilheteria_dataframe = sessao.publico
    total_bilheteria = sum(total_bilheteria_dataframe)
    print(f"O total de bilheteria dos filmes listados foi: {total_bilheteria}")
    return total_bilheteria

questao_1("bilheteria.db")

# Questao 2
2. Qual o filme de maior bilheteria em 2023, por país de origem?

In [ ]:
def questao_2(database: str):
    #carregamos as tabelas necessarias com as funcoes
    filme = carrega_tabela(database, "filme") 
    sessao = carrega_tabela(database, "sessao")

    #a partir da tabela filme, seleccionamos as colunas id, titulo_original e pais_origem
    id_titulo_pais = filme[
        [
            "id",
            "titulo_original",
            "pais_origem"
        ]
    ]
    
    #a partir da tabela sessao, selecionamos as colunas filme_id e publico
    filme_id_publico = sessao[
        [
            "filme_id",
            "publico"
        ]
    ]

    #criamos um diccionarios
    bilheteria_de_cada_filme_por_id = defaultdict(int)    #chave: id de cada filme, valor: total de bilheteria 
    id_de_cada_filme_por_pais = defaultdict(list)         #chave: pais, valor: lista com os id's de cada filme desse pais
    nome_de_cada_filme_por_id = defaultdict(str)          #chave: id de cada filme, valor: nome de cada filme
    filme_de_maior_bilheteria_por_pais = defaultdict(str) #chave: pais, valor: filme com maior bilheteria desse pais

    #colocamos cada pais com seus filmes
    #colocamos cada filme_id com seu filme
    for index in id_titulo_pais.itertuples(index= False):
        filme_id = index.id
        titulo = index.titulo_original
        pais = index.pais_origem
        id_de_cada_filme_por_pais[pais].append(filme_id)
        nome_de_cada_filme_por_id[filme_id] = titulo
    
    #colocamos cada filme_id com seu publico total
    for index in filme_id_publico.itertuples(index=False):
        filme_id = index.filme_id
        publico = index.publico
        bilheteria_de_cada_filme_por_id[filme_id] += publico

    #colocamos cada pais com seu filme de maior bilheteria
    for cada_pais in id_de_cada_filme_por_pais.keys():
        bilheteria = 0
        filme = ""
        
        for cada_filme_id in id_de_cada_filme_por_pais[cada_pais]:
            if bilheteria_de_cada_filme_por_id[cada_filme_id] > bilheteria:
                bilheteria = bilheteria_de_cada_filme_por_id[cada_filme_id]
                filme = nome_de_cada_filme_por_id[cada_filme_id]

        filme_de_maior_bilheteria_por_pais[cada_pais] = filme

    filme_de_maior_bilheteria_por_pais = pd.DataFrame(list(filme_de_maior_bilheteria_por_pais.items()), columns=["pais","filme_de_maior_bilheteria"]).sort_values("pais").reset_index(drop=True)
    return filme_de_maior_bilheteria_por_pais

questao_2("bilheteria.db")

# Questao 3
3. Crie um dataframe com as 100 cidades com maior bilheteria em 2023,
ordenadas de forma decrescente de bilheteria.

In [6]:
def questao_3(database: str):
    #carrgamos as tabelas necessarias
    sala = carrega_tabela(database, "sala")
    complexo = carrega_tabela(database, "complexo")
    sessao = carrega_tabela(database, "sessao")

    """
    da tabela sessao pegamos as colunas sala_id e publico, com a funcao grupby, agrupamos objetos iguais na coluna sala_id
    ouseja, linhas com mesma id serao juntas numa so linha, mas agora temos todos os publicos de uma so id, com todos os publicos
    de uma so id usamos a funcao sum, assim todos os publicos serao adicionados em um so, assim teremos como resultado:
    sala_id, publico_total, reiniciamos o index pois a funcao groupby coloca sala_id automaticamente como index e nao queremos isso
    queremos que sala_id seja tambem uma coluna"""
    sala_id_publico_total = sessao[["sala_id", "publico"]].groupby("sala_id").sum().reset_index()

    """
    da tabela sessao pegamos as colunas, id e complexo, que contem o id de cada sala e o complexo ao qual elas pertencem
    renomeamos a coluna id para sala_id, isso para cosnseguir fazer merge com as duas tabelas  """
    sala_id_compexo = sala[["id","from_complexo"]].rename(columns={"id":"sala_id"})
    
    """
    processo analogo a tabela anterior, agora da tabela complexo seleccionamos apenas id e municipio, renomeando o municipio por
    from_complexo para poder fazer merge com a tabela sala_id_complexo
    """
    complexo_e_cidade = complexo[["id","municipio"]].rename(columns={"id":"from_complexo"})

    """
    fazemos merge de cada tabela que temos nas colunas com mesmo nome, assim conseguimos uma tabela maior com a que conseguiremos
    a funcao groupby com cada cidade, agrupando as cidades semelhantes e somando o publico total de cada cidade
    """
    sala_id_publico_e_complexo = sala_id_publico_total.merge(sala_id_compexo, how="left",on="sala_id")
    sala_id_publico_complexo_e_cidade = sala_id_publico_e_complexo.merge(complexo_e_cidade, how="left",on="from_complexo")
    cidades_bilheteria_total = sala_id_publico_complexo_e_cidade[["municipio","publico"]].groupby("municipio").sum(numeric_only=True)

    #ordenamos em ordem decresente, pegamos as primeiras 100 linhas, reiniciamos o index e renomeamos as colunas
    cem100_cidades_com_mais_bilheteria = cidades_bilheteria_total.sort_values("publico", ascending=False).head(100).reset_index().rename(columns={"publico": "bilheteria_total","municipio":"cidade"})
    
    return cem100_cidades_com_mais_bilheteria
    
questao_3("bilheteria.db")

,cidade,bilheteria_total
0,SÃO PAULO,13310470
1,RIO DE JANEIRO,8496306
2,BRASÍLIA,3272437
3,BELO HORIZONTE,3009414
4,CURITIBA,2965114
...,...,...
95,ARAÇATUBA,247992
96,VITÓRIA DA CONQUISTA,246490
97,PONTA GROSSA,241142
98,LAURO DE FREITAS,240784


# Questao 4
4. Qual o filme com maior bilheteria em cada cidade? Retorne um dataframe
com as colunas CIDADE, FILME e BILHETERIA.

In [ ]:
def questao_4(database):
    sessao = carrega_tabela(database, "sessao")
    filme = carrega_tabela(database, "filme")
    complexo = carrega_tabela(database, "complexo")
    sala = carrega_tabela(database, "sala")


    sala_id__cidade__id = (
        sala[["id", "from_complexo"]]
        .rename(columns={"id":"sala_id","from_complexo":"cidade_id"})
        )
    
    cidade_id__cidade = (
        complexo[["id", "municipio"]]
        .rename(columns={"id":"cidade_id","municipio":"cidade"})
        )

    sala_id__filme_id__publico_total = (
        sessao[["filme_id","sala_id","publico"]]
        .groupby(["sala_id","filme_id"])
        .sum()
        .reset_index()
        )
    
    filme_id__titulo_original = (
        filme[["id", "titulo_original"]]
        .rename(columns={"id":"filme_id"})
        )

    sala_id__filme_id__publico_total__titulo = (
        sala_id__filme_id__publico_total
        .merge(filme_id__titulo_original, how="left", on="filme_id")
        )
    
    sala_id__cidade_id__cidade = (
        sala_id__cidade__id
        .merge(cidade_id__cidade, how="left", on="cidade_id")
        )

    sala_id__titulo_original__publico_total__cidade = (
        sala_id__filme_id__publico_total__titulo[["sala_id","titulo_original","publico"]]
        .merge(sala_id__cidade_id__cidade[["sala_id","cidade"]], how="left", on="sala_id")
        )
    
    cidade__filme__bilheteria = (
        sala_id__titulo_original__publico_total__cidade[["cidade","titulo_original","publico"]]
        .sort_values(by=["cidade","publico"], ascending=[True,False])
        .drop_duplicates(subset="cidade")
        .reset_index(drop=True)
        .rename(columns={"cidade":"CIDADE","titulo_original":"FILME","publico":"BILHETERIA"})
        )

    return  cidade__filme__bilheteria

questao_4("bilheteria.db")


,CIDADE,FILME,BILHETERIA
0,ACARAÚ,BARBIE,3367
1,AGUAÍ,BARBIE,2140
2,ALAGOINHAS,BARBIE,6264
3,ALFENAS,BARBIE,3785
4,ALTAMIRA,BARBIE,2998
...,...,...,...
340,VOLTA REDONDA,BARBIE,6687
341,VOTORANTIM,AVATAR: THE WAY OF WATER,8070
342,VÁRZEA GRANDE,FAST X,6860
343,VÁRZEA PAULISTA,BARBIE,4107
